# Module 1.2: 常見濾波器 (Filters)

## 學習目標

本 notebook 將帶你實作各種常見的影像濾波器：

1. **Box Filter (平均濾波)**：最簡單的平滑濾波
2. **Gaussian Filter (高斯濾波)**：使用高斯函數的平滑濾波
3. **Sobel Filter (邊緣偵測)**：計算影像梯度
4. **Sharpen Filter (銳化濾波)**：增強邊緣

## 前置要求
- 完成 01_convolution.ipynb（了解 2D 卷積）

## 參考資料
- Gonzalez & Woods, Digital Image Processing, Ch. 3-4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
sys.path.append('../../')

# 如果 utils 已正確設置
try:
    from utils.image_io import load_grayscale, save_image
except:
    # 備用方案
    def load_grayscale(path):
        return np.array(Image.open(path).convert('L'), dtype=np.float64)
    def save_image(arr, path):
        Image.fromarray(arr.astype(np.uint8)).save(path)

print("環境設定完成！")

## 複習：從 01_convolution 引入卷積函數

我們先定義上一個 notebook 實作的卷積函數，後續會用來實作各種濾波器。

In [ ]:
def pad_image(image, pad_h, pad_w, mode='constant', value=0):
    """
    對影像進行 zero padding。
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
    pad_h : int, 上下 padding 的 pixel 數
    pad_w : int, 左右 padding 的 pixel 數
    mode : str, 'constant' 或 'reflect'
    value : float, constant mode 時的填充值
    
    Returns
    -------
    padded : np.ndarray, shape (H + 2*pad_h, W + 2*pad_w)
    """
    H, W = image.shape
    new_H = H + 2 * pad_h
    new_W = W + 2 * pad_w
    
    if mode == 'constant':
        padded = np.full((new_H, new_W), value, dtype=image.dtype)
        padded[pad_h:pad_h+H, pad_w:pad_w+W] = image
    elif mode == 'reflect':
        # 使用 np.pad 的 reflect 模式
        padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')
    else:
        raise ValueError(f"Unknown mode: {mode}")
    
    return padded


def conv2d(image, kernel, padding='same'):
    """
    2D 卷積（含 padding 支援）。
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
    kernel : np.ndarray, shape (kH, kW)
    padding : str or int
        'valid': 不 padding
        'same': padding 使輸出與輸入同尺寸
        int: 指定 padding 大小
    
    Returns
    -------
    output : np.ndarray
    """
    kH, kW = kernel.shape
    
    # 計算 padding 大小
    if padding == 'valid':
        pad_h, pad_w = 0, 0
    elif padding == 'same':
        pad_h = (kH - 1) // 2
        pad_w = (kW - 1) // 2
    elif isinstance(padding, int):
        pad_h = pad_w = padding
    else:
        raise ValueError(f"Unknown padding: {padding}")
    
    # 執行 padding
    if pad_h > 0 or pad_w > 0:
        image = pad_image(image, pad_h, pad_w)
    
    H, W = image.shape
    out_H = H - kH + 1
    out_W = W - kW + 1
    
    output = np.zeros((out_H, out_W), dtype=np.float64)
    
    for i in range(out_H):
        for j in range(out_W):
            patch = image[i:i+kH, j:j+kW]
            output[i, j] = np.sum(patch * kernel)
    
    return output

print("卷積函數已載入！")

## 建立測試影像

我們建立一些測試影像來驗證濾波器的效果。

In [ ]:
# 建立一個含有銳利邊緣和雜訊的測試影像
def create_test_image(size=100):
    """建立一個含有各種特徵的測試影像"""
    image = np.zeros((size, size), dtype=np.float64)
    
    # 左上角：白色方塊（測試邊緣）
    image[10:40, 10:40] = 200
    
    # 右上角：漸層（測試平滑效果）
    for i in range(30):
        image[10:40, 60+i] = i * 8
    
    # 下半部：加入雜訊
    noise_region = image[55:95, 10:90].copy()
    noise_region[:] = 100
    noise_region += np.random.randn(*noise_region.shape) * 30
    image[55:95, 10:90] = noise_region
    
    return np.clip(image, 0, 255)

# 建立並顯示測試影像
test_image = create_test_image(100)

plt.figure(figsize=(6, 6))
plt.imshow(test_image, cmap='gray', vmin=0, vmax=255)
plt.title('Test Image')
plt.colorbar()
plt.show()

print(f"影像大小：{test_image.shape}")
print(f"像素範圍：{test_image.min():.1f} ~ {test_image.max():.1f}")

---

# Part 1: Box Filter（平均濾波）

## 概念說明

**Box Filter**（也稱 Mean Filter）是最簡單的平滑濾波器：

- 將每個像素值替換為其鄰域內所有像素的**平均值**
- kernel 中的每個元素值相同，且總和為 1

### 3x3 Box Filter Kernel:

$$
K_{box} = \frac{1}{9} \begin{bmatrix} 
1 & 1 & 1 \\
1 & 1 & 1 \\
1 & 1 & 1 
\end{bmatrix}
$$

### 特點：
- 簡單、計算快速
- 可以去除一些雜訊
- 缺點：會模糊邊緣

In [ ]:
def create_box_kernel(size):
    """
    建立 Box Filter kernel。
    
    Parameters
    ----------
    size : int
        kernel 大小（通常是奇數，如 3, 5, 7）
    
    Returns
    -------
    kernel : np.ndarray, shape (size, size)
        正規化的 box kernel（所有元素和為 1）
    
    數學公式：
    K[i,j] = 1 / (size * size), 對所有 i, j
    """
    # 建立全為 1 的 kernel，然後除以總元素數量以正規化
    kernel = np.ones((size, size), dtype=np.float64)
    kernel = kernel / (size * size)  # 正規化使總和為 1
    
    return kernel

# 測試：建立不同大小的 box kernel
print("3x3 Box Kernel:")
k3 = create_box_kernel(3)
print(k3)
print(f"Sum: {k3.sum():.4f}")

print("\n5x5 Box Kernel:")
k5 = create_box_kernel(5)
print(k5)
print(f"Sum: {k5.sum():.4f}")

In [ ]:
def box_filter(image, size=3):
    """
    對影像套用 Box Filter。
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
        輸入灰階影像
    size : int
        kernel 大小
    
    Returns
    -------
    filtered : np.ndarray
        濾波後的影像
    """
    kernel = create_box_kernel(size)
    filtered = conv2d(image, kernel, padding='same')
    return filtered

# 測試不同大小的 box filter
filtered_3x3 = box_filter(test_image, 3)
filtered_5x5 = box_filter(test_image, 5)
filtered_7x7 = box_filter(test_image, 7)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(filtered_3x3, cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Box 3x3')

axes[2].imshow(filtered_5x5, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Box 5x5')

axes[3].imshow(filtered_7x7, cmap='gray', vmin=0, vmax=255)
axes[3].set_title('Box 7x7')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("觀察：")
print("1. 雜訊區域（下半部）變得比較平滑")
print("2. 邊緣（左上白色方塊）變得模糊")
print("3. kernel 越大，模糊效果越強")

---

# Part 2: Gaussian Filter（高斯濾波）

## 概念說明

**Gaussian Filter** 使用高斯（正態）分布作為權重：

- 中心像素權重最大
- 距離中心越遠，權重越小（呈高斯分布遞減）
- 比 Box Filter 更自然、不會產生振鈴效應

### 2D 高斯函數：

$$
G(x, y) = \frac{1}{2\pi\sigma^2} e^{-\frac{x^2 + y^2}{2\sigma^2}}
$$

其中 $(x, y)$ 是相對於中心的座標，$\sigma$ 是標準差。

### σ (sigma) 的影響：
- **σ 越大**：分布越寬，模糊效果越強
- **σ 越小**：分布越窄，模糊效果越弱

### Kernel 大小的選擇：
- 通常設為 $\text{size} = \lceil 6\sigma \rceil$ 或更大
- 確保能涵蓋大部分的高斯分布（超過 99.7%）

In [ ]:
def create_gaussian_kernel(size, sigma):
    """
    建立 Gaussian kernel。
    
    Parameters
    ----------
    size : int
        kernel 大小（應為奇數）
    sigma : float
        高斯分布的標準差
    
    Returns
    -------
    kernel : np.ndarray, shape (size, size)
        正規化的高斯 kernel（所有元素和為 1）
    
    實作步驟：
    1. 建立座標網格（以中心為原點）
    2. 計算每個位置的高斯值
    3. 正規化使總和為 1
    """
    # 確保 size 是奇數
    if size % 2 == 0:
        size += 1
    
    # 建立座標（中心為 0）
    half = size // 2
    x = np.arange(-half, half + 1)
    y = np.arange(-half, half + 1)
    X, Y = np.meshgrid(x, y)
    
    # 計算高斯值
    # G(x,y) = exp(-(x^2 + y^2) / (2 * sigma^2))
    gaussian = np.exp(-(X**2 + Y**2) / (2 * sigma**2))
    
    # 正規化使總和為 1
    kernel = gaussian / gaussian.sum()
    
    return kernel

# 測試：建立不同 sigma 的 Gaussian kernel
print("Gaussian Kernel (size=5, sigma=1.0):")
g1 = create_gaussian_kernel(5, 1.0)
print(np.round(g1, 4))
print(f"Sum: {g1.sum():.4f}")

print("\nGaussian Kernel (size=5, sigma=2.0):")
g2 = create_gaussian_kernel(5, 2.0)
print(np.round(g2, 4))
print(f"Sum: {g2.sum():.4f}")

In [ ]:
# 視覺化 Gaussian kernel
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

sigmas = [0.5, 1.0, 2.0]
for i, sigma in enumerate(sigmas):
    kernel = create_gaussian_kernel(11, sigma)
    im = axes[i].imshow(kernel, cmap='hot')
    axes[i].set_title(f'σ = {sigma}')
    plt.colorbar(im, ax=axes[i])

plt.suptitle('Gaussian Kernels with Different Sigma')
plt.tight_layout()
plt.show()

# 1D 切面圖
plt.figure(figsize=(10, 4))
for sigma in [0.5, 1.0, 1.5, 2.0]:
    kernel = create_gaussian_kernel(15, sigma)
    center = kernel.shape[0] // 2
    profile = kernel[center, :]
    x = np.arange(len(profile)) - center
    plt.plot(x, profile, label=f'σ = {sigma}')

plt.xlabel('Distance from center')
plt.ylabel('Weight')
plt.title('1D Cross-section of Gaussian Kernels')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def gaussian_filter(image, sigma, size=None):
    """
    對影像套用 Gaussian Filter。
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
        輸入灰階影像
    sigma : float
        高斯標準差
    size : int or None
        kernel 大小，若為 None 則自動計算為 ceil(6*sigma) | 1
    
    Returns
    -------
    filtered : np.ndarray
        濾波後的影像
    """
    if size is None:
        # 自動計算 kernel 大小
        size = int(np.ceil(6 * sigma))
        if size % 2 == 0:
            size += 1  # 確保是奇數
    
    kernel = create_gaussian_kernel(size, sigma)
    filtered = conv2d(image, kernel, padding='same')
    return filtered

# 測試不同 sigma 的 Gaussian filter
filtered_s05 = gaussian_filter(test_image, sigma=0.5)
filtered_s10 = gaussian_filter(test_image, sigma=1.0)
filtered_s20 = gaussian_filter(test_image, sigma=2.0)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(filtered_s05, cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Gaussian σ=0.5')

axes[2].imshow(filtered_s10, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Gaussian σ=1.0')

axes[3].imshow(filtered_s20, cmap='gray', vmin=0, vmax=255)
axes[3].set_title('Gaussian σ=2.0')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Box Filter vs Gaussian Filter 比較
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Row 1: 濾波結果
axes[0, 0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Original')

axes[0, 1].imshow(box_filter(test_image, 5), cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title('Box 5x5')

axes[0, 2].imshow(gaussian_filter(test_image, 1.0, 5), cmap='gray', vmin=0, vmax=255)
axes[0, 2].set_title('Gaussian 5x5, σ=1.0')

# Row 2: Kernel 視覺化
axes[1, 0].text(0.5, 0.5, 'Kernel\nComparison', ha='center', va='center', fontsize=16)
axes[1, 0].axis('off')

im1 = axes[1, 1].imshow(create_box_kernel(5), cmap='hot')
axes[1, 1].set_title('Box Kernel 5x5')
plt.colorbar(im1, ax=axes[1, 1])

im2 = axes[1, 2].imshow(create_gaussian_kernel(5, 1.0), cmap='hot')
axes[1, 2].set_title('Gaussian Kernel 5x5')
plt.colorbar(im2, ax=axes[1, 2])

for ax in axes[0]:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("觀察：")
print("1. Box Filter 權重均勻分布")
print("2. Gaussian Filter 中心權重高，邊緣權重低")
print("3. Gaussian Filter 的模糊效果更自然")

---

# Part 3: Sobel Filter（邊緣偵測）

## 概念說明

**Sobel Filter** 用於計算影像的梯度（gradient），是邊緣偵測的基礎。

### 原理：
- 邊緣 = 像素值**變化劇烈**的地方
- 梯度 = 變化率（一階導數）
- 使用有限差分近似導數

### Sobel Kernels：

水平梯度（偵測垂直邊緣）：
$$
G_x = \begin{bmatrix} 
-1 & 0 & 1 \\
-2 & 0 & 2 \\
-1 & 0 & 1 
\end{bmatrix}
$$

垂直梯度（偵測水平邊緣）：
$$
G_y = \begin{bmatrix} 
-1 & -2 & -1 \\
0 & 0 & 0 \\
1 & 2 & 1 
\end{bmatrix}
$$

### 梯度 Magnitude 和 Direction：

$$
\text{Magnitude} = \sqrt{G_x^2 + G_y^2}
$$

$$
\text{Direction} = \arctan\left(\frac{G_y}{G_x}\right)
$$

In [ ]:
# 定義 Sobel kernels
SOBEL_X = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], dtype=np.float64)

SOBEL_Y = np.array([
    [-1, -2, -1],
    [ 0,  0,  0],
    [ 1,  2,  1]
], dtype=np.float64)

print("Sobel X (偵測垂直邊緣):")
print(SOBEL_X)

print("\nSobel Y (偵測水平邊緣):")
print(SOBEL_Y)

# 視覺化 kernels
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

im1 = axes[0].imshow(SOBEL_X, cmap='RdBu', vmin=-2, vmax=2)
axes[0].set_title('Sobel X')
for i in range(3):
    for j in range(3):
        axes[0].text(j, i, f'{SOBEL_X[i,j]:.0f}', ha='center', va='center', fontsize=12)
plt.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(SOBEL_Y, cmap='RdBu', vmin=-2, vmax=2)
axes[1].set_title('Sobel Y')
for i in range(3):
    for j in range(3):
        axes[1].text(j, i, f'{SOBEL_Y[i,j]:.0f}', ha='center', va='center', fontsize=12)
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

In [ ]:
def sobel_edge_detection(image):
    """
    使用 Sobel 運算子計算梯度。
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
        輸入灰階影像
    
    Returns
    -------
    gradient_x : np.ndarray
        水平方向梯度
    gradient_y : np.ndarray
        垂直方向梯度
    magnitude : np.ndarray
        梯度強度 sqrt(gx^2 + gy^2)
    direction : np.ndarray
        梯度方向（弧度）
    
    實作步驟：
    1. 用 Sobel X kernel 卷積得到 Gx
    2. 用 Sobel Y kernel 卷積得到 Gy
    3. 計算 magnitude = sqrt(Gx^2 + Gy^2)
    4. 計算 direction = arctan2(Gy, Gx)
    """
    # Step 1: 計算水平梯度
    gradient_x = conv2d(image, SOBEL_X, padding='same')
    
    # Step 2: 計算垂直梯度
    gradient_y = conv2d(image, SOBEL_Y, padding='same')
    
    # Step 3: 計算梯度強度
    magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    
    # Step 4: 計算梯度方向
    direction = np.arctan2(gradient_y, gradient_x)
    
    return gradient_x, gradient_y, magnitude, direction

# 測試 Sobel 邊緣偵測
gx, gy, mag, dir_ = sobel_edge_detection(test_image)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Row 1
axes[0, 0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Original')

im1 = axes[0, 1].imshow(gx, cmap='RdBu')
axes[0, 1].set_title('Gradient X (Gx)')
plt.colorbar(im1, ax=axes[0, 1])

im2 = axes[0, 2].imshow(gy, cmap='RdBu')
axes[0, 2].set_title('Gradient Y (Gy)')
plt.colorbar(im2, ax=axes[0, 2])

# Row 2
axes[1, 0].axis('off')

im3 = axes[1, 1].imshow(mag, cmap='gray')
axes[1, 1].set_title('Gradient Magnitude')
plt.colorbar(im3, ax=axes[1, 1])

im4 = axes[1, 2].imshow(dir_, cmap='hsv', vmin=-np.pi, vmax=np.pi)
axes[1, 2].set_title('Gradient Direction')
plt.colorbar(im4, ax=axes[1, 2])

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("觀察：")
print("1. Gx 在垂直邊緣（左右變化）有反應")
print("2. Gy 在水平邊緣（上下變化）有反應")
print("3. Magnitude 結合兩者，顯示所有邊緣強度")
print("4. Direction 顯示邊緣的方向（顏色代表角度）")

In [ ]:
# 練習：其他邊緣偵測 kernels

# Prewitt operator (比 Sobel 更簡單)
PREWITT_X = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1]
], dtype=np.float64)

PREWITT_Y = np.array([
    [-1, -1, -1],
    [ 0,  0,  0],
    [ 1,  1,  1]
], dtype=np.float64)

# Roberts cross operator (最小的邊緣偵測器，2x2)
ROBERTS_X = np.array([
    [1,  0],
    [0, -1]
], dtype=np.float64)

ROBERTS_Y = np.array([
    [0,  1],
    [-1, 0]
], dtype=np.float64)

def compute_edge_magnitude(image, kernel_x, kernel_y):
    """使用給定的 kernels 計算邊緣強度"""
    gx = conv2d(image, kernel_x, padding='same')
    gy = conv2d(image, kernel_y, padding='same')
    return np.sqrt(gx**2 + gy**2)

# 比較不同邊緣偵測器
sobel_mag = compute_edge_magnitude(test_image, SOBEL_X, SOBEL_Y)
prewitt_mag = compute_edge_magnitude(test_image, PREWITT_X, PREWITT_Y)
roberts_mag = compute_edge_magnitude(test_image, ROBERTS_X, ROBERTS_Y)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(sobel_mag, cmap='gray')
axes[1].set_title('Sobel')

axes[2].imshow(prewitt_mag, cmap='gray')
axes[2].set_title('Prewitt')

axes[3].imshow(roberts_mag, cmap='gray')
axes[3].set_title('Roberts')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("比較：")
print("1. Sobel 對雜訊更強健（有 2x 加權）")
print("2. Prewitt 更均勻但對雜訊較敏感")
print("3. Roberts 最小最快，但容易受雜訊影響")

---

# Part 4: Sharpen Filter（銳化濾波）

## 概念說明

**Sharpen Filter** 用於增強影像的邊緣和細節。

### 原理：

$$
\text{Sharpened} = \text{Original} + k \cdot \text{Edges}
$$

其中 Edges 可以用 Laplacian 運算子偵測。

### Laplacian 運算子（二階導數）：

$$
\nabla^2 f = \frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2}
$$

### 常見 Laplacian Kernels：

4-鄰域：
$$
L_4 = \begin{bmatrix} 
0 & 1 & 0 \\
1 & -4 & 1 \\
0 & 1 & 0 
\end{bmatrix}
$$

8-鄰域：
$$
L_8 = \begin{bmatrix} 
1 & 1 & 1 \\
1 & -8 & 1 \\
1 & 1 & 1 
\end{bmatrix}
$$

### 合併的 Sharpen Kernel：

$$
K_{sharpen} = \begin{bmatrix} 
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0 
\end{bmatrix}
$$

這等於 Identity + (-Laplacian)

In [ ]:
# 定義 Laplacian kernels
LAPLACIAN_4 = np.array([
    [0,  1, 0],
    [1, -4, 1],
    [0,  1, 0]
], dtype=np.float64)

LAPLACIAN_8 = np.array([
    [1,  1, 1],
    [1, -8, 1],
    [1,  1, 1]
], dtype=np.float64)

# Identity kernel (什麼都不做)
IDENTITY = np.array([
    [0, 0, 0],
    [0, 1, 0],
    [0, 0, 0]
], dtype=np.float64)

# Sharpen kernel = Identity - Laplacian
SHARPEN = IDENTITY - LAPLACIAN_4

print("Laplacian (4-neighbor):")
print(LAPLACIAN_4)

print("\nSharpen Kernel:")
print(SHARPEN)

# 視覺化
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

kernels = [LAPLACIAN_4, LAPLACIAN_8, SHARPEN]
titles = ['Laplacian 4-neighbor', 'Laplacian 8-neighbor', 'Sharpen']

for ax, kernel, title in zip(axes, kernels, titles):
    im = ax.imshow(kernel, cmap='RdBu', vmin=-8, vmax=8)
    ax.set_title(title)
    for i in range(3):
        for j in range(3):
            ax.text(j, i, f'{kernel[i,j]:.0f}', ha='center', va='center', fontsize=12)
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

In [ ]:
def laplacian_edge_detection(image, mode='4'):
    """
    使用 Laplacian 運算子偵測邊緣。
    
    Parameters
    ----------
    image : np.ndarray
        輸入影像
    mode : str
        '4' 或 '8' 鄰域
    
    Returns
    -------
    edges : np.ndarray
        邊緣影像
    """
    if mode == '4':
        kernel = LAPLACIAN_4
    else:
        kernel = LAPLACIAN_8
    
    edges = conv2d(image, kernel, padding='same')
    return edges


def sharpen_image(image, strength=1.0):
    """
    銳化影像。
    
    Parameters
    ----------
    image : np.ndarray
        輸入影像
    strength : float
        銳化強度，1.0 為預設，>1.0 更強
    
    Returns
    -------
    sharpened : np.ndarray
        銳化後的影像
    
    實作方式：
    sharpened = original + strength * (original - blurred)
    或使用 sharpen kernel
    """
    # 方法 1：使用 sharpen kernel
    # kernel = IDENTITY - strength * LAPLACIAN_4 / 4
    
    # 方法 2：使用 unsharp masking
    # sharpened = original + strength * (original - blurred)
    
    # 這裡使用方法 2，更直觀
    blurred = gaussian_filter(image, sigma=1.0)
    sharpened = image + strength * (image - blurred)
    
    # 確保值在有效範圍內
    sharpened = np.clip(sharpened, 0, 255)
    
    return sharpened


def sharpen_with_kernel(image, strength=1.0):
    """
    使用 kernel 直接銳化影像。
    """
    # 調整銳化強度的 kernel
    kernel = np.array([
        [0, -strength, 0],
        [-strength, 1 + 4*strength, -strength],
        [0, -strength, 0]
    ], dtype=np.float64)
    
    sharpened = conv2d(image, kernel, padding='same')
    return np.clip(sharpened, 0, 255)

In [ ]:
# 測試 Laplacian 邊緣偵測
lap4 = laplacian_edge_detection(test_image, '4')
lap8 = laplacian_edge_detection(test_image, '8')

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

im1 = axes[1].imshow(lap4, cmap='RdBu')
axes[1].set_title('Laplacian 4-neighbor')
plt.colorbar(im1, ax=axes[1])

im2 = axes[2].imshow(lap8, cmap='RdBu')
axes[2].set_title('Laplacian 8-neighbor')
plt.colorbar(im2, ax=axes[2])

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("注意：")
print("1. Laplacian 對雜訊非常敏感")
print("2. 8-鄰域版本包含對角線方向")
print("3. 邊緣有正負值（表示亮→暗或暗→亮）")

In [ ]:
# 測試銳化效果
sharp1 = sharpen_image(test_image, strength=0.5)
sharp2 = sharpen_image(test_image, strength=1.0)
sharp3 = sharpen_image(test_image, strength=2.0)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(sharp1, cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Sharpen (strength=0.5)')

axes[2].imshow(sharp2, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Sharpen (strength=1.0)')

axes[3].imshow(sharp3, cmap='gray', vmin=0, vmax=255)
axes[3].set_title('Sharpen (strength=2.0)')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("觀察：")
print("1. 邊緣變得更清晰")
print("2. strength 越大，邊緣越明顯，但雜訊也會被放大")

---

# Part 5: 綜合練習

## 練習 1: 先模糊再偵測邊緣

**目的**：了解 Gaussian smoothing 在邊緣偵測中的作用（這是 Canny 的第一步）

In [ ]:
def gaussian_then_sobel(image, sigma=1.0):
    """
    先做 Gaussian smoothing，再做 Sobel 邊緣偵測。
    
    這是 Canny 邊緣偵測的前兩個步驟！
    
    Parameters
    ----------
    image : np.ndarray
        輸入影像
    sigma : float
        Gaussian 平滑的標準差
    
    Returns
    -------
    magnitude : np.ndarray
        梯度強度
    direction : np.ndarray
        梯度方向
    """
    # Step 1: Gaussian smoothing 降低雜訊
    smoothed = gaussian_filter(image, sigma=sigma)
    
    # Step 2: Sobel 邊緣偵測
    _, _, magnitude, direction = sobel_edge_detection(smoothed)
    
    return magnitude, direction

# 比較有無 smoothing 的差異
_, _, mag_no_smooth, _ = sobel_edge_detection(test_image)
mag_smooth_1, _ = gaussian_then_sobel(test_image, sigma=1.0)
mag_smooth_2, _ = gaussian_then_sobel(test_image, sigma=2.0)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(mag_no_smooth, cmap='gray')
axes[1].set_title('Sobel (No Smoothing)')

axes[2].imshow(mag_smooth_1, cmap='gray')
axes[2].set_title('Gaussian(σ=1) + Sobel')

axes[3].imshow(mag_smooth_2, cmap='gray')
axes[3].set_title('Gaussian(σ=2) + Sobel')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("觀察：")
print("1. 雜訊區域（下半部）的假邊緣大幅減少")
print("2. 真正的邊緣仍然保留")
print("3. σ 越大，雜訊去除越多，但邊緣也會變模糊")

## 練習 2: 建立自己的 Filter 函數庫

整合所有實作的 filter 成一個完整的模組。

In [ ]:
class ImageFilters:
    """
    影像濾波器集合。
    
    這個類別整合了所有我們實作的濾波器。
    """
    
    # 預定義的 kernels
    SOBEL_X = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float64)
    SOBEL_Y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float64)
    LAPLACIAN_4 = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]], dtype=np.float64)
    LAPLACIAN_8 = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], dtype=np.float64)
    
    @staticmethod
    def box_kernel(size):
        """建立 Box kernel"""
        return np.ones((size, size), dtype=np.float64) / (size * size)
    
    @staticmethod
    def gaussian_kernel(size, sigma):
        """建立 Gaussian kernel"""
        half = size // 2
        x = np.arange(-half, half + 1)
        X, Y = np.meshgrid(x, x)
        gaussian = np.exp(-(X**2 + Y**2) / (2 * sigma**2))
        return gaussian / gaussian.sum()
    
    @staticmethod
    def sharpen_kernel(strength=1.0):
        """建立 Sharpen kernel"""
        return np.array([
            [0, -strength, 0],
            [-strength, 1 + 4*strength, -strength],
            [0, -strength, 0]
        ], dtype=np.float64)
    
    @classmethod
    def apply(cls, image, filter_type, **kwargs):
        """
        統一的濾波器介面。
        
        Parameters
        ----------
        image : np.ndarray
            輸入影像
        filter_type : str
            濾波器類型：'box', 'gaussian', 'sobel', 'laplacian', 'sharpen'
        **kwargs : dict
            濾波器參數
        
        Returns
        -------
        filtered : np.ndarray
            濾波後的影像
        """
        if filter_type == 'box':
            size = kwargs.get('size', 3)
            kernel = cls.box_kernel(size)
            return conv2d(image, kernel, padding='same')
        
        elif filter_type == 'gaussian':
            sigma = kwargs.get('sigma', 1.0)
            size = kwargs.get('size', int(np.ceil(6 * sigma)) | 1)
            kernel = cls.gaussian_kernel(size, sigma)
            return conv2d(image, kernel, padding='same')
        
        elif filter_type == 'sobel':
            gx = conv2d(image, cls.SOBEL_X, padding='same')
            gy = conv2d(image, cls.SOBEL_Y, padding='same')
            return np.sqrt(gx**2 + gy**2)
        
        elif filter_type == 'laplacian':
            mode = kwargs.get('mode', '4')
            kernel = cls.LAPLACIAN_4 if mode == '4' else cls.LAPLACIAN_8
            return conv2d(image, kernel, padding='same')
        
        elif filter_type == 'sharpen':
            strength = kwargs.get('strength', 1.0)
            kernel = cls.sharpen_kernel(strength)
            result = conv2d(image, kernel, padding='same')
            return np.clip(result, 0, 255)
        
        else:
            raise ValueError(f"Unknown filter type: {filter_type}")

# 測試整合的類別
print("使用 ImageFilters 類別：")

# 各種濾波
box_result = ImageFilters.apply(test_image, 'box', size=5)
gauss_result = ImageFilters.apply(test_image, 'gaussian', sigma=1.5)
sobel_result = ImageFilters.apply(test_image, 'sobel')
sharp_result = ImageFilters.apply(test_image, 'sharpen', strength=1.0)

fig, axes = plt.subplots(1, 5, figsize=(20, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(box_result, cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Box 5x5')

axes[2].imshow(gauss_result, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Gaussian σ=1.5')

axes[3].imshow(sobel_result, cmap='gray')
axes[3].set_title('Sobel Edge')

axes[4].imshow(sharp_result, cmap='gray', vmin=0, vmax=255)
axes[4].set_title('Sharpen')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

## 練習 3: 濾波器組合

嘗試不同的濾波器組合來達成特定目標。

In [ ]:
def denoise_and_sharpen(image, denoise_sigma=1.0, sharpen_strength=1.0):
    """
    去雜訊並銳化的組合流程。
    
    流程：
    1. Gaussian smoothing 去雜訊
    2. Sharpen 恢復邊緣細節
    
    Parameters
    ----------
    image : np.ndarray
        輸入影像
    denoise_sigma : float
        去雜訊的 Gaussian sigma
    sharpen_strength : float
        銳化強度
    
    Returns
    -------
    result : np.ndarray
        處理後的影像
    """
    # Step 1: 去雜訊
    denoised = ImageFilters.apply(image, 'gaussian', sigma=denoise_sigma)
    
    # Step 2: 銳化
    result = ImageFilters.apply(denoised, 'sharpen', strength=sharpen_strength)
    
    return result

# 測試
processed = denoise_and_sharpen(test_image, denoise_sigma=1.0, sharpen_strength=0.8)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original (with noise)')

axes[1].imshow(gaussian_filter(test_image, 1.0), cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Denoised only')

axes[2].imshow(processed, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Denoised + Sharpened')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("觀察：")
print("1. 只去雜訊會讓影像變模糊")
print("2. 組合去雜訊 + 銳化可以保持邊緣清晰度")

---

# 總結

本 notebook 實作了以下濾波器：

| 濾波器 | 功能 | Kernel 特點 |
|--------|------|-------------|
| Box Filter | 平滑（模糊） | 均勻權重 |
| Gaussian Filter | 平滑（更自然） | 中心高、邊緣低 |
| Sobel Filter | 邊緣偵測（一階導數） | 方向性 |
| Laplacian | 邊緣偵測（二階導數） | 各向同性 |
| Sharpen | 增強邊緣 | 中心增強、周圍減弱 |

## 關鍵概念

1. **平滑濾波器**（Box, Gaussian）用於去雜訊，但會模糊邊緣
2. **邊緣偵測濾波器**（Sobel, Laplacian）用於找出影像中的邊緣
3. **銳化濾波器**用於增強細節，但會放大雜訊
4. 實際應用中常需要組合多種濾波器

## 下一步

- 在 Module 2 中，我們會實作完整的 **Canny 邊緣偵測**，它結合了：
  - Gaussian smoothing
  - Sobel gradient
  - Non-maximum suppression
  - Hysteresis thresholding

In [ ]:
# 最終驗證：確認所有函數都能正常運作
print("=== 函數驗證 ===")

test_funcs = [
    ("create_box_kernel(5)", lambda: create_box_kernel(5)),
    ("create_gaussian_kernel(5, 1.0)", lambda: create_gaussian_kernel(5, 1.0)),
    ("box_filter(image, 3)", lambda: box_filter(test_image, 3)),
    ("gaussian_filter(image, 1.0)", lambda: gaussian_filter(test_image, 1.0)),
    ("sobel_edge_detection(image)", lambda: sobel_edge_detection(test_image)),
    ("laplacian_edge_detection(image)", lambda: laplacian_edge_detection(test_image)),
    ("sharpen_image(image, 1.0)", lambda: sharpen_image(test_image, 1.0)),
    ("gaussian_then_sobel(image, 1.0)", lambda: gaussian_then_sobel(test_image, 1.0)),
    ("ImageFilters.apply(image, 'box')", lambda: ImageFilters.apply(test_image, 'box')),
    ("ImageFilters.apply(image, 'gaussian')", lambda: ImageFilters.apply(test_image, 'gaussian')),
    ("ImageFilters.apply(image, 'sobel')", lambda: ImageFilters.apply(test_image, 'sobel')),
    ("ImageFilters.apply(image, 'sharpen')", lambda: ImageFilters.apply(test_image, 'sharpen')),
]

for name, func in test_funcs:
    try:
        result = func()
        print(f"✓ {name}")
    except Exception as e:
        print(f"✗ {name}: {e}")

print("\n所有函數驗證完成！")